In [8]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr # pyright: ignore[reportMissingTypeStubs, reportUnknownVariableType]
from typing import Dict, List, Tuple
from model_ranking.results import results_to_arrays, get_summary_results
from model_ranking.correlation import scores_to_rank, permutation_test_kendall_tau, permutation_test_spearman_rho

In [ ]:
selected_augmentations: Dict[str, List[str]] = {
    "none": [],
    "DO": [ "a005", "a01", "a02", "a03", "a04", "a05"],
}
source_models = {
    "BBBC039": "BC_model",
    "Go-Nuclear": "GN_model",
    "HeLaNuc": "HN_model",
    "Hoechst": "Hst_model",
    "S_BIAD895": "895_model",
    "S_BIAD1410": "1410_model",
}
per_target_norms: Dict[str, List[Tuple[float, float]]] = {
    "BBBC039": [(5, 98)],
    "DSB2018": [(5, 98)],
    "Hoechst": [(5, 98)],
    "S_BIAD634": [(5, 98)],
    "S_BIAD895": [(5, 98)],
}

result_folders = {
    "BBBC039": "exp1",
    "DSB2018": "exp1",
    "Hoechst": "exp1",
    "S_BIAD634": "exp1",
    "S_BIAD895": "exp1",
}
consis_keys = {
    "BBBC039": "HD_consis",
    "DSB2018": "HD_consis",
    "Hoechst": "HD_consis",
    "S_BIAD634": "HD_consis",
    "S_BIAD895": "HD_consis",
}
target_datasets = [
    "BBBC039",
    "DSB2018",
    "Hoechst",
    "S_BIAD634",
    "S_BIAD895",
]
source_datasets = [
    "BBBC039",
    "Go-Nuclear",
    "HeLaNuc",
    "Hoechst",
    "S_BIAD895",
    "S_BIAD1410",
]

base_seg_dir = "/path/to/results/directory"
invert_consis = True
invert_perf = False
rank_ascending=False
perf_tolerance=0
consis_tolerance=0
perf_key = "f1_eval"

In [ ]:
kendall_per_transfer = np.zeros((len(target_datasets), len(selected_augmentations["DO"])))
k_pval_per_transfer = np.zeros((len(target_datasets), len(selected_augmentations["DO"])))
spearman_per_transfer = np.zeros((len(target_datasets), len(selected_augmentations["DO"])))
sp_pval_per_transfer = np.zeros((len(target_datasets), len(selected_augmentations["DO"])))
pearson_per_transfer = np.zeros((len(target_datasets), len(selected_augmentations["DO"])))
p_pval_per_transfer = np.zeros((len(target_datasets), len(selected_augmentations["DO"])))
for i, target in enumerate(target_datasets):
    if (target == "S_BIAD895") and (target in source_datasets):
        sources = source_datasets.copy()
        sources.remove(target)
    else:
        sources = source_datasets
    consis_str, perf_str, NA_perf = get_summary_results(
        source_data=sources, 
        target_data=[target], 
        source_models=source_models, 
        selected_augmentations=selected_augmentations,
        selected_norms=per_target_norms,
        consis_keys=consis_keys,
        perf_key="f1_eval",
        per_target_norms=True,
        result_folders=result_folders,
        approach="feature_perturbation_consistency",
        consis_postfix= "median",
        perf_postfix="median",
        base_seg_dir=base_seg_dir,
    )
    consis_scores, NA_perf_scores = results_to_arrays(
        consis_str, NA_perf, "DO", len(selected_augmentations["DO"])
    )
    if invert_consis:
        consis_scores = 1 - consis_scores
    if invert_perf:
        NA_perf_scores = 1 - NA_perf_scores

    target_df = pd.DataFrame()
    target_df['aug_strength'] = selected_augmentations["DO"]
    ken_scores = np.zeros(consis_scores.shape[1])
    ken_pvalue = np.zeros(consis_scores.shape[1])
    spear_scores = np.zeros(consis_scores.shape[1])
    spear_pval = np.zeros(consis_scores.shape[1])
    pearsons_scores = np.zeros(consis_scores.shape[1])
    pearsons_pval = np.zeros(consis_scores.shape[1])
    for j in range(consis_scores.shape[1]):
        #print(f"Consis Tolerance {consis_tolerance}")
        #print(f"Aug strength {selected_augmentations['DO'][j]}, Target {target}")
        #print(f"performance scores { NA_perf_scores}")
        ranked_scores = scores_to_rank(NA_perf_scores, ascending=rank_ascending, tolerance=perf_tolerance)
        #print(f"ranked scores {ranked_scores}")
        #print(f"consistency scores {consis_scores[:, j]}")
        ranked_consis = scores_to_rank(consis_scores[:, j],ascending=rank_ascending, tolerance=consis_tolerance)
        #print(f"ranked consistency {ranked_consis}")
        #pearson, spearman, kendall = prediction_correlation(ranked_scores, consis_scores[:, i])
        pearson, pearson_pval = pearsonr(NA_perf_scores, consis_scores[:,j])
        spearman, spearmans_pval = permutation_test_spearman_rho(ranked_scores, ranked_consis)
        ken, pval = permutation_test_kendall_tau(ranked_scores, ranked_consis)
        #pearson, spearman, kendall = prediction_correlation(ranked_scores, ranked_consis)
        #print(f"{selected_augmentations['DO'][j]}: p={pearson:.3f}, s={spearman:.3f}, k{ken:.3f}")
        
        #print(f"{selected_augmentations['DO'][j]}: ken{ken:.3f}, pvalue={pval:.3f}")
        #print("\n")
        ken_scores[j] = ken
        ken_pvalue[j] = pval
        spear_scores[j] = spearman
        spear_pval[j] = spearmans_pval
        pearsons_scores[j] = pearson
        pearsons_pval[j] = pearson_pval
    target_df["kendall tau"] = ken_scores
    target_df["kendall p-value"] = ken_pvalue
    target_df["spearman rho"] = spear_scores
    target_df["spearman p-value"] = spear_pval
    target_df["pearson r"] = pearsons_scores
    target_df["pearson p-value"] = pearsons_pval
    kendall_per_transfer[i] = ken_scores
    k_pval_per_transfer[i] = ken_pvalue
    spearman_per_transfer[i] = spear_scores
    sp_pval_per_transfer[i] = spear_pval
    pearson_per_transfer[i] = pearsons_scores
    p_pval_per_transfer[i] = pearsons_pval

Source: BBBC039


100%|██████████| 1/1 [00:00<00:00, 12.70it/s]


Source: Go-Nuclear


100%|██████████| 1/1 [00:00<00:00, 11.36it/s]


Source: HeLaNuc


...

100%|██████████| 1/1 [00:00<00:00,  8.86it/s]


Source: Hoechst


100%|██████████| 1/1 [00:00<00:00, 25.61it/s]


Source: S_BIAD1410


100%|██████████| 1/1 [00:00<00:00,  6.30it/s]


In [15]:
aug_strength = "a02"
idx = selected_augmentations["DO"].index(aug_strength)
df = pd.DataFrame(
    {"kt": kendall_per_transfer[:,idx], "kt pval": k_pval_per_transfer[:,idx], "s rho": spearman_per_transfer[:,idx], "s rho pval": sp_pval_per_transfer[:,idx], "pr": pearson_per_transfer[:,idx], "pr pval": p_pval_per_transfer[:,idx]},
)
df["targets"] = target_datasets
# make targets index
df = df.set_index("targets")
#add multi index to dataframe with a single index named Nuclei that covers all rows
df.index = pd.MultiIndex.from_product([["Nuclei"], df.index], names=["Task", "targets"])
print(df)

                        kt   kt pval     s rho  s rho pval        pr   pr pval
Task   targets                                                                
Nuclei BBBC039    0.866667  0.023976  0.942857    0.013986  0.935037  0.006193
       DSB2018    0.733333  0.059940  0.828571    0.055944  0.901286  0.014136
       Hoechst    0.866667  0.025974  0.942857    0.017982  0.997827  0.000007
       S_BIAD634  1.000000  0.001998  1.000000    0.003996  0.852120  0.031186
       S_BIAD895  0.600000  0.267732  0.700000    0.229770  0.684240  0.202603
